<a href="https://colab.research.google.com/github/feerose111/eSewa-AI-Internship/blob/main/transaction_categorization_using_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [6]:
import pandas as pd

try:
    df = pd.read_csv('/content/data1.csv', encoding='utf-8')  # Try with utf-8 first
except UnicodeDecodeError:
    # If utf-8 fails, try other encodings
    try:
        df = pd.read_csv('/content/data1.csv', encoding='latin-1')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv('/content/data1.csv', encoding='iso-8859-1')
        except UnicodeDecodeError:
            # If none of the above work, try 'cp1252'
            df = pd.read_csv('/content/data1.csv', encoding='cp1252')

In [22]:
df.head()


,Description,Category,Sub_Category
0,Arby's,Food,FastFood_Restaurants
1,Burger King,Food,FastFood_Restaurants
2,Carl's Jr.,Food,FastFood_Restaurants
3,Chick-fil-A,Food,FastFood_Restaurants
4,Chipotle Mexican Grill,Food,FastFood_Restaurants


In [23]:
df.tail()

,Description,Category,Sub_Category
3465,RoxBerry Juice,Food,FastFood_Restaurants
3466,Café Rio,Food,FastFood_Restaurants
3467,My Tire Guys,Auto,Maintenance
3468,Google Google Storage,Subscriptions_Memberships,Entertainment
3469,Protonmail,Subscriptions_Memberships,Entertainment


In [19]:

df.describe()

,Description,Category,Sub_Category
count,3470,3470,3470
unique,3368,15,42
top,Gerber Baby Girls' 4-Pack Sleep N' Play,Electronics,Tools
freq,4,768,261


In [13]:
df.shape


(3470, 3)

In [15]:
df.isna().sum()

,0
Description,0
Category,0
Sub_Category,0


In [25]:
df.value_counts()

,,,count
Description,Category,Sub_Category,
Dewalt DCBL720P1 20V MAX Cordless Leaf Blower,Home,Tools,4
Gerber Baby Girls' 4-Pack Sleep N' Play,Clothes,Baby_Clothes,4
Jack in the Box,Food,FastFood_Restaurants,3
Simple Joys by Carter's Baby Girls' 4-Pack Pant,Clothes,Baby_Clothes,3
Dunkin' Donuts,Food,FastFood_Restaurants,3
...,...,...,...
Frigidaire FGSC2335TF Side-by-Side Refrigerator,Home,Furniture_Appliances,1
Frontier Airlines,Vacation,Flights,1
Frontier Communications,Utilities,Internet,1


In [29]:
label_encoder = LabelEncoder()
df["category_encoded"] = label_encoder.fit_transform(df["Category"])

In [35]:
df.value_counts('Category')

,count
Category,
Electronics,768
Home,577
Clothes,466
Entertainment,407
Personal_Care,401
Vacation,199
Auto,178
Food,137
Utilities,133


In [48]:
category_mapping = {
    'Personal_Care': 'Health',
    'Personal Care': 'Health',
    'Medical': 'Health',
    'Entertainment': 'Entertainment',
    'Vacation': 'Entertainment',
    'Utilities': 'Bills',
    'Debt': 'Bills',
    'Savings': 'Bills',
    'Subscriptions_Memberships': 'Bills',
    'Kids': 'Bills'
}
df['New_Category'] = df['Category'].map(category_mapping).fillna(df['Category'])


In [49]:

df.value_counts('New_Category')

,count
New_Category,
Electronics,768
Entertainment,606
Home,577
Clothes,466
Health,404
Bills,334
Auto,178
Food,137


In [69]:
label_encoder = LabelEncoder()
df["category_encoded"] = label_encoder.fit_transform(df["New_Category"])

In [74]:
df.value_counts('category_encoded')

,count
category_encoded,
3,768
4,606
7,577
2,466
6,404
1,334
0,178
5,137


In [60]:
df['combined_text'] = df['Description'] + ' ' + df['Sub_Category']


In [61]:
print(df['combined_text'])

0                       Arby's FastFood_Restaurants
1                  Burger King FastFood_Restaurants
2                   Carl's Jr. FastFood_Restaurants
3                  Chick-fil-A FastFood_Restaurants
4       Chipotle Mexican Grill FastFood_Restaurants
                           ...                     
3465            RoxBerry Juice FastFood_Restaurants
3466                  Café Rio FastFood_Restaurants
3467                       My Tire Guys Maintenance
3468            Google Google Storage Entertainment
3469                       Protonmail Entertainment
Name: combined_text, Length: 3470, dtype: object


In [62]:
#converting the text data to feature vector
vectorizer = TfidfVectorizer()

In [63]:
feature_vectors = vectorizer.fit_transform(df['combined_text'])

In [64]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18518 stored elements and shape (3470, 4295)>
  Coords	Values
  (0, 415)	0.8567464297488714
  (0, 1469)	0.5157378744212627
  (1, 1469)	0.4318649755244836
  (1, 718)	0.665718873599859
  (1, 2098)	0.608531859682114
  (2, 1469)	0.3916540939126342
  (2, 779)	0.6506178105160791
  (2, 2055)	0.6506178105160791
  (3, 1469)	0.3916540939126342
  (3, 859)	0.6506178105160791
  (3, 1499)	0.6506178105160791
  (4, 1469)	0.3282871672447875
  (4, 863)	0.5453523435426534
  (4, 2416)	0.5453523435426534
  (4, 1760)	0.5453523435426534
  (5, 1469)	0.39010337354232905
  (5, 1162)	0.6720479659076879
  (5, 3750)	0.6294210748538958
  (6, 1469)	0.41146976042246963
  (6, 1229)	0.6835356984149397
  (6, 2848)	0.6028860466541552
  (7, 1469)	0.3916540939126342
  (7, 1281)	0.6506178105160791
  (7, 1230)	0.6506178105160791
  (8, 1469)	0.3964355378540496
  :	:
  (3462, 4091)	0.49049747092539336
  (3462, 3013)	0.5500828588276827
  (3463, 3675)	0.39174642445406

In [75]:
X = feature_vectors
Y = df['category_encoded']

In [76]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18518 stored elements and shape (3470, 4295)>
  Coords	Values
  (0, 415)	0.8567464297488714
  (0, 1469)	0.5157378744212627
  (1, 1469)	0.4318649755244836
  (1, 718)	0.665718873599859
  (1, 2098)	0.608531859682114
  (2, 1469)	0.3916540939126342
  (2, 779)	0.6506178105160791
  (2, 2055)	0.6506178105160791
  (3, 1469)	0.3916540939126342
  (3, 859)	0.6506178105160791
  (3, 1499)	0.6506178105160791
  (4, 1469)	0.3282871672447875
  (4, 863)	0.5453523435426534
  (4, 2416)	0.5453523435426534
  (4, 1760)	0.5453523435426534
  (5, 1469)	0.39010337354232905
  (5, 1162)	0.6720479659076879
  (5, 3750)	0.6294210748538958
  (6, 1469)	0.41146976042246963
  (6, 1229)	0.6835356984149397
  (6, 2848)	0.6028860466541552
  (7, 1469)	0.3916540939126342
  (7, 1281)	0.6506178105160791
  (7, 1230)	0.6506178105160791
  (8, 1469)	0.3964355378540496
  :	:
  (3462, 4091)	0.49049747092539336
  (3462, 3013)	0.5500828588276827
  (3463, 3675)	0.39174642445406

In [77]:
print(Y)

0       5
1       5
2       5
3       5
4       5
       ..
3465    5
3466    5
3467    0
3468    1
3469    1
Name: category_encoded, Length: 3470, dtype: int64


In [78]:
# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [82]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [83]:
y_pred = model.predict(X_test)

In [84]:
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

               precision    recall  f1-score   support

         Auto       1.00      1.00      1.00        35
        Bills       0.99      0.93      0.96        71
      Clothes       1.00      1.00      1.00        98
  Electronics       0.98      0.98      0.98       140
Entertainment       0.95      1.00      0.98       119
         Food       1.00      0.97      0.98        32
       Health       1.00      0.99      0.99        87
         Home       1.00      1.00      1.00       112

     accuracy                           0.99       694
    macro avg       0.99      0.98      0.99       694
 weighted avg       0.99      0.99      0.99       694



In [87]:
new_data = ["Uber ride home", "Amazon electronics", "Netflix subscription"]  # sample unseen data
new_data_vec = vectorizer.transform(new_data)
predictions = model.predict(new_data_vec)

for desc, pred in zip(new_data, predictions):
    print(f"{desc} ➜ Predicted Category: {pred}")


Uber ride home ➜ Predicted Category: 7
Amazon electronics ➜ Predicted Category: 3
Netflix subscription ➜ Predicted Category: 4
